In [17]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import traceback

In [3]:
df = pd.read_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [27]:
df[df['press'] == 'Activistpost.com']['url'].iloc[0]

'https://www.activistpost.com/2017/11/video-photo-evidence-now-even-easily-faked-fabricated.html'

In [28]:
Activistpost = df[df['press'] == 'Activistpost.com']
Activistpost

,source,author,title,description,url,urlToImage,publishedAt,content,press
1692,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Video and Photo Evidence Now Even More Easily ...,By Makia Freeman Photo evidence and video evid...,https://www.activistpost.com/2017/11/video-pho...,https://www.activistpost.com/wp-content/upload...,2017-11-15T18:00:45Z,By Makia Freeman Photo evidence and video evid...,Activistpost.com
2060,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Bank Admits Fiat Currencies Are Failing and Cr...,By Shaun Bradley As the transition towards a b...,https://www.activistpost.com/2017/11/bank-admi...,NaN,2017-11-16T17:15:26Z,By Shaun Bradley As the transition towards a b...,Activistpost.com
3034,"{'id': None, 'name': 'Activistpost.com'}",ActivistPost,WATCH LIVE! Vin Armani Talks Bitcoin Cash & Au...,"Vin discusses developments in Bitcoin Cash, br...",https://www.activistpost.com/2017/11/watch-liv...,NaN,2017-11-20T16:51:55Z,The Vin Armani Show airs live on the video bel...,Activistpost.com
4685,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Financial Crisis – Imminent and Inevitable?,By Chris Kanthan It’s never fun to warn people...,https://www.activistpost.com/2017/11/financial...,https://www.activistpost.com/wp-content/upload...,2017-11-26T01:44:44Z,By Chris Kanthan It’s never fun to warn people...,Activistpost.com
5283,"{'id': None, 'name': 'Activistpost.com'}",ActivistPost,WATCH LIVE: Understanding The Differences Betw...,By Vin Armani Watch the livestream of the Vin ...,https://www.activistpost.com/2017/11/watch-liv...,https://www.activistpost.com/wp-content/upload...,2017-11-27T17:34:18Z,By Vin Armani Watch the livestream of the Vin ...,Activistpost.com
...,...,...,...,...,...,...,...,...,...
744159,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,WEF Pushes Criminalizing “climate inaction”,By The Counter Signal The World Economic Forum...,https://www.activistpost.com/2023/09/wef-pushe...,https://www.activistpost.com/wp-content/upload...,2023-09-29T19:01:54Z,By The Counter Signal\r\nThe World Economic Fo...,Activistpost.com
744385,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Trudeau Government Moves to Regulate Podcasts,By Mike Campbell The Trudeau government’s rece...,https://www.activistpost.com/2023/09/trudeau-g...,https://www.activistpost.com/wp-content/upload...,2023-09-30T17:01:36Z,By Mike Campbell\r\nThe Trudeau governments re...,Activistpost.com
744386,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,The White House’s ‘Misinformation’ Pressure Ca...,By Aaron Kheriaty Iam one of five private plai...,https://www.activistpost.com/2023/09/the-white...,https://www.activistpost.com/wp-content/upload...,2023-09-30T17:09:08Z,By Aaron Kheriaty\r\nIam one of five private p...,Activistpost.com
744388,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Is the Money in Your Checking Account Yours or...,By Jonathan Newman When Silicon Valley Bank an...,https://www.activistpost.com/2023/09/is-the-mo...,https://www.activistpost.com/wp-content/upload...,2023-09-30T17:22:24Z,By Jonathan Newman\r\nWhen Silicon Valley Bank...,Activistpost.com


In [32]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import re

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            content = []
            for element in soup.select('.entry-content'):
                content.append(element.get_text())

            cleaned_content = ' '.join(content)
            contents.append(cleaned_content)
            # print(element)
        except Exception as e:
            print(f"Error occurred: {e}")
            traceback.print_exc()  # 예외의 traceback 정보를 출력합니다.
            contents.append('')

    return contents


In [24]:
url = 'https://www.ccn.com/bitcoin-keeping-awake-night-bank-canada-governor/'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
soup

<!DOCTYPE html>

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" rel="stylesheet"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function

In [25]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")
            
            selected_elements = soup.select('.ccn-content')
            
            # soup.select('.ccn-content')가 빈 리스트를 반환할 경우 에러 메시지 출력
            if not selected_elements:
                print(f"No elements found for URL {url}.")
                contents.append('')
                continue

            content = []
            for element in selected_elements:
                content.append(element.get_text())
                print(element)

            cleaned_content = ' '.join(content)
            contents.append(cleaned_content)
        except Exception as e:
            print(f"Error occurred while processing URL {url}: {e}")
            contents.append('')

    return contents



In [88]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
            classes = set()
            for tag in soup.find_all(True, {'class': True}):
                for class_name in tag['class']:
                    if re.match(r'col-\w+-offset-1', class_name):
                        classes.add(class_name)

            # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
            content = []
            for class_name in classes:
                content.extend(soup.select(f".{class_name}"))
            
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
            contents.append(cleaned_content)
        except:
            contents.append('')

    return contents


In [34]:
head = Activistpost.head()

In [36]:
# Adding the content to the DataFrame
Activistpost['content_full'] = get_contents(Activistpost)
Activistpost

100%|██████████| 8311/8311 [5:51:27<00:00,  2.54s/it]   
<ipython-input-36-b5b39142ccb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Activistpost['content_full'] = get_contents(Activistpost)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
1692,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Video and Photo Evidence Now Even More Easily ...,By Makia Freeman Photo evidence and video evid...,https://www.activistpost.com/2017/11/video-pho...,https://www.activistpost.com/wp-content/upload...,2017-11-15T18:00:45Z,By Makia Freeman Photo evidence and video evid...,Activistpost.com,\nBy Makia Freeman\nPhoto evidence and video e...
2060,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Bank Admits Fiat Currencies Are Failing and Cr...,By Shaun Bradley As the transition towards a b...,https://www.activistpost.com/2017/11/bank-admi...,NaN,2017-11-16T17:15:26Z,By Shaun Bradley As the transition towards a b...,Activistpost.com,\nBy Shaun Bradley\nAs the transition towards ...
3034,"{'id': None, 'name': 'Activistpost.com'}",ActivistPost,WATCH LIVE! Vin Armani Talks Bitcoin Cash & Au...,"Vin discusses developments in Bitcoin Cash, br...",https://www.activistpost.com/2017/11/watch-liv...,NaN,2017-11-20T16:51:55Z,The Vin Armani Show airs live on the video bel...,Activistpost.com,\n\nThe Vin Armani Show airs live on the video...
4685,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Financial Crisis – Imminent and Inevitable?,By Chris Kanthan It’s never fun to warn people...,https://www.activistpost.com/2017/11/financial...,https://www.activistpost.com/wp-content/upload...,2017-11-26T01:44:44Z,By Chris Kanthan It’s never fun to warn people...,Activistpost.com,\nBy Chris Kanthan\nIt’s never fun to warn peo...
5283,"{'id': None, 'name': 'Activistpost.com'}",ActivistPost,WATCH LIVE: Understanding The Differences Betw...,By Vin Armani Watch the livestream of the Vin ...,https://www.activistpost.com/2017/11/watch-liv...,https://www.activistpost.com/wp-content/upload...,2017-11-27T17:34:18Z,By Vin Armani Watch the livestream of the Vin ...,Activistpost.com,\n\nBy Vin Armani\nWatch the livestream of the...
...,...,...,...,...,...,...,...,...,...,...
744159,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,WEF Pushes Criminalizing “climate inaction”,By The Counter Signal The World Economic Forum...,https://www.activistpost.com/2023/09/wef-pushe...,https://www.activistpost.com/wp-content/upload...,2023-09-29T19:01:54Z,By The Counter Signal\r\nThe World Economic Fo...,Activistpost.com,\nBy The Counter Signal\nThe World Economic Fo...
744385,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Trudeau Government Moves to Regulate Podcasts,By Mike Campbell The Trudeau government’s rece...,https://www.activistpost.com/2023/09/trudeau-g...,https://www.activistpost.com/wp-content/upload...,2023-09-30T17:01:36Z,By Mike Campbell\r\nThe Trudeau governments re...,Activistpost.com,\nBy Mike Campbell\nThe Trudeau government’s r...
744386,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,The White House’s ‘Misinformation’ Pressure Ca...,By Aaron Kheriaty Iam one of five private plai...,https://www.activistpost.com/2023/09/the-white...,https://www.activistpost.com/wp-content/upload...,2023-09-30T17:09:08Z,By Aaron Kheriaty\r\nIam one of five private p...,Activistpost.com,\nBy Aaron Kheriaty\nI am one of five private ...
744388,"{'id': None, 'name': 'Activistpost.com'}",Activist Post,Is the Money in Your Checking Account Yours or...,By Jonathan Newman When Silicon Valley Bank an...,https://www.activistpost.com/2023/09/is-the-mo...,https://www.activistpost.com/wp-content/upload...,2023-09-30T17:22:24Z,By Jonathan Newman\r\nWhen Silicon Valley Bank...,Activistpost.com,\nBy Jonathan Newman\nWhen Silicon Valley Bank...


In [87]:
head['url'].iloc[0]

'https://www.reuters.com/markets/us/biden-administration-unveils-new-crypto-tax-reporting-rules-2023-08-25/'

In [29]:
# with1 = PRNewswire[PRNewswire['content_full'] != '']
without['url'].iloc[10]

'https://mediablog.prnewswire.com/2020/07/15/9-unique-finance-news-sites-that-are-worth-investing-your-time-in/'

In [103]:
[Reuters[Reuters['content_full'] == '']['url']][0]

31        https://ca.reuters.com/article/technologyNews/...
181       https://in.reuters.com/article/us-global-marke...
646       https://ca.reuters.com/article/businessNews/id...
674       https://in.reuters.com/article/global-markets/...
720       https://af.reuters.com/article/investingNews/i...
                                ...                        
580106    http://rmb.reuters.com/rmd/rss/item/tag:reuter...
583541    https://www.reuters.com/investigates/special-r...
587344    https://www.reuters.com/investigates/special-r...
593161    http://rmb.reuters.com/rmd/rss/item/tag:reuter...
683392    https://www.reuters.com/investigates/special-r...
Name: url, Length: 1777, dtype: object

In [114]:
empty_rows = Reuters[Reuters['content_full'] == '']

empty_rows['url'].iloc[4]

'https://af.reuters.com/article/investingNews/idAFKBN1DD0Q1-OZABS'

In [24]:
coindesk

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
56,"{'id': None, 'name': 'CoinDesk'}",Nikhilesh De,More Charges Filed Against Trader Who Used Bit...,A Philadelphia day trader was formally indicte...,https://www.coindesk.com/day-trader-formally-i...,https://media.coindesk.com/uploads/2017/11/Unt...,2017-11-09T23:00:16Z,New charges have been filed against a Philadel...,CoinDesk,New charges have been filed against a Philadel...
141,"{'id': None, 'name': 'CoinDesk'}",Alyssa Hertig,Ethereum to ICOs: You're Doing It Wrong,Ethereum devs have harsh words for many ICO la...,https://www.coindesk.com/ethereum-icos-youre-w...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T09:00:52Z,"""Grotesque"" might not be the word you'd think ...",CoinDesk,"""Grotesque"" might not be the word you'd think ..."
167,"{'id': None, 'name': 'CoinDesk'}",Rachel Rose O'leary,Bitcoin Classic Announces Closure In Wake of S...,Proposed bitcoin scaling solution Bitcoin Clas...,https://www.coindesk.com/bitcoin-classic-annou...,https://media.coindesk.com/uploads/2017/11/clo...,2017-11-10T11:22:12Z,Proposed bitcoin scaling solution Bitcoin Clas...,CoinDesk,A group of developers coding under the banner ...
175,"{'id': None, 'name': 'CoinDesk'}",Sujha Sundararajan,US Treasury Secretary: We're Looking 'Carefull...,"The U.S. Secretary of the Treasury, Steven Mnu...",https://www.coindesk.com/us-treasury-secretary...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T12:05:14Z,"The U.S. Secretary of the Treasury, Steven Mnu...",CoinDesk,"The U.S. Secretary of the Treasury, Steven Mnu..."
204,"{'id': None, 'name': 'CoinDesk'}",Omkar Godbole,Bull Grip Weakens As Bitcoin Price Breaks Belo...,Following the setting of a new record high of ...,https://www.coindesk.com/bull-grip-weakens-bit...,https://media.coindesk.com/uploads/2017/11/Cli...,2017-11-10T13:31:26Z,Bitcoin remains on the back foot ahead of the ...,CoinDesk,Bitcoin remains on the back foot ahead of the ...
...,...,...,...,...,...,...,...,...,...,...
743942,"{'id': None, 'name': 'CoinDesk'}",Jamie Crawley,Gemini to Cease Offering Crypto Services in Ne...,The exchange said it is taking the step due to...,https://www.coindesk.com/policy/2023/09/29/gem...,https://www.coindesk.com/resizer/5JOs1BQJ79kue...,2023-09-29T12:47:11Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,Cryptocurrency exchange Gemini will stop offer...
743960,"{'id': None, 'name': 'CoinDesk'}","Lyllah Ledesma, Omkar Godbole",First Mover Americas: Circle Argues Stablecoin...,The latest price moves in bitcoin (BTC) and cr...,https://www.coindesk.com/markets/2023/09/29/fi...,https://www.coindesk.com/resizer/iDYQEx_XIxIfl...,2023-09-29T13:10:20Z,This article originally appeared in First Move...,CoinDesk,This article originally appeared in First Move...
744030,"{'id': None, 'name': 'CoinDesk'}",Danny Chong,Where Liquid Staking Meets Tokenization,"The crypto industry is changing rapidly, and l...",https://www.coindesk.com/consensus-magazine/20...,https://www.coindesk.com/resizer/Y0wciIP16Z0Yw...,2023-09-29T15:00:21Z,Liquid staking has the potential to be the bui...,CoinDesk,Liquid staking has the potential to be the bui...
744047,"{'id': None, 'name': 'CoinDesk'}",Lyllah Ledesma,Valkyrie Backs Off Ether Futures Purchases Unt...,The asset manager Thursday said it had already...,https://www.coindesk.com/markets/2023/09/29/va...,https://www.coindesk.com/resizer/psYGVjbtB9QL0...,2023-09-29T15:24:23Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,Valkyrie Friday morning said it will not purch...


In [113]:
Reuters.to_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\Reuters_14.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content